In [10]:
#but du code: récupérer la dataset et la preprocessing
from google.colab import drive

drive.mount('/content/drive/')

%cd /content/drive/My Drive/ENSC/3A/S9/AlphaGo/
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/ENSC/3A/S9/AlphaGo
 alpha_all.pdf		 Goban.py	        NNV5.ipynb
 alphago.pdf		 Labels_next_move.npy   __pycache__
 alphago_zero.pdf	 Labels.npy	        samples-9x9.json.gz
 Features_multiple.npy	 Labels_value.npy       training_1
 Features.npy		 NNV3.ipynb
'fiche alphago.gdoc'	 NNV4.ipynb


In [11]:
import os
import urllib
import gzip
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Dense, Flatten, Add
from tensorflow.keras.activations import linear, relu, tanh
import numpy as np
import random
import Goban
import keras
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate

In [12]:
#aide: https://stackoverflow.com/questions/62285475/why-am-i-having-this-error-typeerror-failed-to-convert-object-of-type-class

In [13]:
def get_raw_data_go():
    ''' Returns the set of samples from the local file or download it if it does not exists'''

    raw_samples_file = "samples-9x9.json.gz"

    if not os.path.isfile(raw_samples_file):
        print("File", raw_samples_file, "not found, I am downloading it...", end="")
        urllib.request.urlretrieve("https://www.labri.fr/perso/lsimon/ia-inge2/samples-9x9.json.gz", "samples-9x9.json.gz")
        print(" Done")

    with gzip.open("samples-9x9.json.gz") as fz:
        data = json.loads(fz.read().decode("utf-8"))
    return data

def fulfill_board(list_moves):
    board=Goban.Board()
    list_col_letters=['A','B','C','D','E','F','G','H','I']
    for i in range(len(list_moves)):
        board.push(b.name_to_flat(list_moves[i]))
    return board


def get_all_boards_from_game(list_moves):
    global_board=np.zeros([9,9,len(list_moves)])
    list_col_letters=['A','B','C','D','E','F','G','H','I']
    current_board=Goban.Board()
    for i in range(len(list_moves)):
        #print(list_moves[i])
        #print(i)
        #print(current_board.name_to_flat(list_moves[i])) #erreur à 14
        try:
            current_board.push(current_board.name_to_flat(list_moves[i]))
        except KeyError:
            print('ERROR')
            break
            #print(list_moves[i]) #A9
            #print(name_to_flat(list_moves[i]))
            #print(current_board.name_to_flat(list_moves[i])) #72
            #print(list_moves)
        
        global_board[:,:,i]=current_board._board.reshape(9,9)
    return global_board



In [ ]:
def generate_single_data(dataset):

    rand_sample=random.randint(0,len(dataset)-1)
    first_game=dataset[rand_sample]
    list_moves=first_game.get('list_of_moves')


    nb_boards_of_interest=4

    #on prend 4 boards
    boards=get_all_boards_from_game(list_moves)
    #on prend un board au hasard qui contienne assez de boards précédemment
    num_board_end=random.randint(5,len(list_moves))

    boards_interest=boards[:,:,num_board_end-nb_boards_of_interest:num_board_end] #à modifier, faut juste qu'on puiss récupérer les 4 boards précédents
    #print(boards_interest.shape)
    white_boards=np.zeros([9,9,nb_boards_of_interest])
    black_boards=np.zeros([9,9,nb_boards_of_interest])
    j=0
    for j in range(boards_interest.shape[2]):
        random_board=boards_interest[:,:,j]
        black_board=np.zeros([9,9])

        black_board=np.where(random_board==1,1,black_board)
        black_boards[:,:,j]=black_board


        white_board=np.zeros([9,9])

        white_board=np.where(random_board==2,2,white_board)
        white_boards[:,:,j]=white_board

        j+=1
    
    #c'est le joueur noir qui joue en premier
    #donc noir joue le coup 0
    #donc noir a joué les coups pairs
    #donc blanc joue le prochain coup
    if(num_board_end%2==0):
        player='white'
    else:
        player='black'
        
    if(player=='white'): #si c'est pair, c'est aux blancs de jouer
        next_player=np.zeros([9,9])
        value=(first_game.get('white_wins'))/100
    else:
        next_player=np.ones([9,9])
        value=first_game.get('black_wins')/100
    #on récupère ensuite l'entier qui correspond à la position dans la liste du coup joué
    next_move=num_board_end + 1
    #print(next_move)
    #a=input()


    #déjà on prend un board
    input_data=np.zeros([9,9,9])
    #on prend 
    input_data[:,:,0:nb_boards_of_interest]=white_boards
    input_data[:,:,nb_boards_of_interest:nb_boards_of_interest*2]=black_boards
    input_data[:,:,nb_boards_of_interest*2]=next_player
    #faut aussi return la proba de gagner
    return (input_data, value,next_move)


dataset=get_raw_data_go()
#sample, value=generate_single_data(dataset)
#print(sample.shape)
#print(sample[:,2,:])
#ça a l'air bon, on a la bonne shape
#print(value)

#on crée un tableau 
nb_samples=100000
tab_features_multiple=np.zeros([nb_samples,9,9,9])
tab_labels_value=np.zeros(nb_samples)
tab_labels_next_move=np.zeros(nb_samples)

for i in range(nb_samples):
    sample=np.zeros([9,9,9])
    value=0
    sample, value, next_move=generate_single_data(dataset)
    
    tab_features_multiple[i,:,:,:]=sample
    tab_labels_value[i]=value
    tab_labels_next_move[i] =next_move





#il y a des erreurs même dans 50 samples,  key error: 24

#le problème c'est qu'il push plusieurs fois le même move

#on voit sur l'exemple suivant qu'effectivement, on push plusieurs fois le même move
#je vois pas trop comment c'est possible
#on save features et labels
np.save('Features_multiple.npy',tab_features_multiple)
np.save('Labels_value.npy',tab_labels_value)
np.save('Labels_next_move.npy',tab_labels_next_move)

In [ ]:
features=np.load('Features_multiple.npy')
labels_value=np.load('Labels_value.npy')
labels_next_move=np.load('Labels_next_move.npy')
print(features.shape)
print(labels_next_move.shape)
print(labels_value.shape)

final_label_moves=np.zeros([labels_next_move.shape[0], 83])
for i in range(final_label_moves.shape[0]):
  j=labels_next_move[i]
  j=int(j)
  print(i)
  print(j)
  final_label_moves[i,j]=1

In [18]:
#residual_tower

visible=Input(shape=(9,9,9))
conv1=Conv2D(256,(3,3), strides=(1, 1), input_shape=(9,9,9), padding='same') (visible)
norm1= BatchNormalization() (conv1)
output= Activation(relu) (norm1)

residual_tower=Model(inputs=visible, outputs=output)
#print(residual_tower.output_shape)

In [19]:
#residual block

visible=Input(shape=residual_tower.output_shape[1:])
conv1=Conv2D(256,(3,3), strides=(1, 1), input_shape=residual_tower.output_shape[1:], padding='same') (visible)
norm1= BatchNormalization() (conv1)
act1= Activation(relu) (norm1)
conv2=Conv2D(256,(3,3), strides=(1, 1), padding='same') (act1)
norm2= BatchNormalization() (conv2)
#residual layer
act2=Activation(relu, trainable=False) (norm2)
conv3=Conv2D(256,(3,3), strides=(1, 1), padding='same') (act2)
act3=Activation(relu) (conv3)
conv4=Conv2D(256,(3,3), strides=(1, 1), padding='same') (act3)
residual=Add() ([conv4, act2])
#act5=Activation(relu) (residual)
#res1=Residual(256,(9,9)) (norm2)
output=Activation(relu) (residual)
residual_block=Model(inputs=visible, outputs=output)

print(residual_block.output_shape)

(None, 9, 9, 256)


In [20]:
#value head
visible=Input(shape=residual_block.output_shape[1:])
conv1= Conv2D(1,(1,1), strides=(1, 1), input_shape=residual_block.output_shape[1:], padding='same') (visible)
norm1= BatchNormalization() (conv1)
act1= Activation(relu) (norm1)
flat1= Flatten() (act1)
dense1= Dense(256, activation='relu') (flat1)
output= Dense(1, activation='tanh') (dense1)

value_head=Model(inputs=visible, outputs=output)
print(value_head.output_shape)

visible=Input(shape=(9,9,9))
output_residual_tower=residual_tower(visible)
for i in range(19):
  if(i==0): #on prend la sortie de residual tower
    output_residual_block=residual_block(output_residual_tower)
  else:
    output_residual_block=residual_block(output_residual_block)
output_value=value_head(output_residual_block)
value_model=Model(inputs=visible, outputs=output_value)
print(value_model.output_shape)


value_model.compile(
#loss=my_loss_fn,
#loss=['mean_squared_error', 'categorical_crossentropy'],
loss='mean_squared_error',
optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
metrics=['accuracy'])

#value_model.fit(features,  labels_value, batch_size=64, epochs=1)

(None, 1)
(None, 1)


In [23]:
#policy_head

visible=Input(shape=residual_block.output_shape[1:])
conv1=Conv2D(2,(1,1), strides=(1, 1), input_shape=residual_block.output_shape[1:], padding='same') (visible)
norm1= BatchNormalization() (conv1)
act1= Activation(relu) (norm1)
flat1= Flatten() (act1)
output=Dense(83, activation='softmax') (flat1)

policy_head=Model(inputs=visible, outputs=output)

visible=Input(shape=(9,9,9))
output_residual_tower=residual_tower(visible)
for i in range(19):
  if(i==0): #on prend la sortie de residual tower
    output_residual_block=residual_block(output_residual_tower)
  else:
    output_residual_block=residual_block(output_residual_block)
output_value=policy_head(output_residual_block)
policy_model=Model(inputs=visible, outputs=output_value)
print(policy_model.output_shape)

policy_model.compile(
#loss=my_loss_fn,
#loss=['mean_squared_error', 'categorical_crossentropy'],
loss='categorical_crossentropy',
optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
metrics=['accuracy']
#metrics=tf.keras.metrics.SparseCategoricalAccuracy()
)

#je suis censé prédire une distribution de probabilités avec un vecteur
#de 82 composantes. je dois donc avoir softmax

#policy_model.fit(features,  final_label_moves, batch_size=64, epochs=1)

(None, 83)


In [ ]:
#global model with 2 outputs
visible=Input(shape=(9,9,9))
output_residual_tower=residual_tower(visible)
for i in range(19):
  if(i==0): #on prend la sortie de residual tower
    output_residual_block=residual_block(output_residual_tower)
  else:
    output_residual_block=residual_block(output_residual_block)
output_value=value_head(output_residual_block)
output_policy=policy_head(output_residual_block)
global_model=Model(inputs=visible, outputs=[output_value, output_policy])
print(global_model.output_shape)

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)



global_model.compile(
#loss=my_loss_fn,
loss=['mean_squared_error', 'categorical_crossentropy'],
#loss='sparse_categorical_crossentropy',
optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
metrics=['accuracy'])



global_model.fit(features,  [labels_value, final_label_moves], batch_size=64, epochs=50, callbacks=[cp_callback])

[(None, 1), (None, 83)]
Epoch 1/50
1563/1563 [==============================] - ETA: 0s - loss: 3.0142 - model_9_loss: 0.1417 - model_14_loss: 2.8725 - model_9_accuracy: 0.1776 - model_14_accuracy: 0.1887
Epoch 00001: saving model to training_1/cp.ckpt
1563/1563 [==============================] - 987s 629ms/step - loss: 3.0142 - model_9_loss: 0.1417 - model_14_loss: 2.8725 - model_9_accuracy: 0.1776 - model_14_accuracy: 0.1887
Epoch 2/50
1563/1563 [==============================] - ETA: 0s - loss: 1.5251 - model_9_loss: 0.1209 - model_14_loss: 1.4042 - model_9_accuracy: 0.2189 - model_14_accuracy: 0.5018
Epoch 00002: saving model to training_1/cp.ckpt
1563/1563 [==============================] - 983s 629ms/step - loss: 1.5251 - model_9_loss: 0.1209 - model_14_loss: 1.4042 - model_9_accuracy: 0.2189 - model_14_accuracy: 0.5018
Epoch 3/50
1563/1563 [==============================] - ETA: 0s - loss: 1.2150 - model_9_loss: 0.1162 - model_14_loss: 1.0988 - model_9_accuracy: 0.2359 - model_1